
# Optimization Problem with Gurobi

The Swatt company produces electric batteries of three types (**A, B, and G**).
- Two of them (**B and G**) use **copper**, while the other two (**A and B**) use **nickel**.
- The company can purchase copper at **5 €/kg** and nickel at **7 €/kg**.
- However, the supplier can provide at most **200 kg of copper** and **200 kg of nickel**.

Each battery type is sold at:
- **A**: 45 € per box
- **B**: 40 € per box
- **G**: 30 € per box

The batteries are produced in two different plants (**I1 and I2**).
The tables below show the amount of copper and nickel required to produce one box of each battery type, along with the labor costs per box.

## **Resource Consumption and Labor Costs**

### **Plant I1**
| Battery | Copper (kg/box) | Nickel (kg/box) | Labor Cost (€) |
|---------|----------------|-----------------|---------------|
| A       | -              | 2               | 12            |
| B       | 1              | 3               | 6             |
| G       | 2              | -               | 4             |

### **Plant I2**
| Battery | Copper (kg/box) | Nickel (kg/box) | Labor Cost (€) |
|---------|----------------|-----------------|---------------|
| A       | -              | 4               | 8             |
| B       | 2              | 2               | 10            |
| G       | 5              | -               | 3             |

## **Production Constraints**
Each plant has a **maximum production capacity** (boxes per battery type):

| Plant | A (max boxes) | B (max boxes) | G (max boxes) |
|-------|--------------|--------------|--------------|
| I1    | 30           | 30           | 10           |
| I2    | 40           | 20           | 20           |

Additionally, the production quantities must satisfy the following conditions:

- The number of boxes of **A** must be at least **twice** the number of boxes of **B**.
- The number of boxes of **A** must **not exceed** the number of boxes of **G**.

## **Optimization Goal**
Determine the production plan that **maximizes the total profit** while satisfying all constraints.



In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Setting the model
m = gp.Model("BatteryProduction")

# Types of batteries and plants
batteries = ['A','B','G']
plants = ['I1','I2']

# Selling prices
price = {'A': 45, 'B': 40, 'G': 30}

# Material costs
cost_copper = 5
cost_nickel = 7

# Material limits
max_copper = 200
max_nickel = 200

# Work capacity
capacity = {('I1', 'A'): 30, ('I1', 'B'): 30, ('I1', 'G'): 10,
            ('I2', 'A'): 40, ('I2', 'B'): 20, ('I2', 'G'): 20}

# Material usage
copper_usage = {('I1', 'A'): 0, ('I1', 'B'): 1, ('I1', 'G'): 2,
            ('I2', 'A'): 0, ('I2', 'B'): 2, ('I2', 'G'): 5}

nickel_usage = {('I1', 'A'): 2, ('I1', 'B'): 3, ('I1', 'G'): 0,
            ('I2', 'A'): 4, ('I2', 'B'): 2, ('I2', 'G'): 0}

# Labor cost
labor_cost = {('I1', 'A'): 12, ('I1', 'B'): 6, ('I1', 'G'): 4,
            ('I2', 'A'): 8, ('I2', 'B'): 10, ('I2', 'G'): 3}

# Decision variables
x = m.addVars(plants, batteries, vtype=GRB.INTEGER, name="x")

# Objective function: maximize the profit
m.setObjective(
    sum(price[b]*x[j, b] - (cost_copper*copper_usage[j, b] + cost_nickel * nickel_usage[j, b] + labor_cost[j, b]) * x[j, b]
        for j in plants for b in batteries), GRB.MAXIMIZE)

# Resources constrains
m.addConstr(sum(copper_usage[j, b] * x[j, b] for j in plants for b in batteries)<= max_copper, "CopperLimit")
m.addConstr(sum(nickel_usage[j, b] * x[j, b] for j in plants for b in batteries)<= max_nickel, "NickelLimit")


# Productivity constrains
for j in plants:
    for b in batteries:
        m.addConstr(x[j, b] <= capacity[j, b], f"Capacity_{j}_{b}")

# Relational constrains
m.addConstr(sum(x[j, 'A'] for j in plants) >= 2 * sum(x[j, 'B'] for j in plants), "ABRelation")
m.addConstr(sum(x[j, 'A'] for j in plants) <= sum(x[j, 'G'] for j in plants), "AGRelation")

# Solving
m.optimize()

# Output cleaning
if m.status == GRB.Status.OPTIMAL:
    print("Optimal solution:")
    for j in plants:
        for b in batteries:
            print(f"{b} in {j}: {x[j, b].x}")
    print(f"Maximal profit: {m.objVal}")